# My first NLP Project 
В проекте буду использовать библиотек nltk, sklearn, numpy, pandas, matplotlib, seaborn

1. Токенизация

In [37]:
import pandas as pd
import numpy as np

In [165]:
df = pd.read_csv("labeled.csv")
df

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0
...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0


# 1. Стоп слова, Токенизация текста, Стемминг и Лиммитизация
Используем nltk

In [166]:
import nltk
import re
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
nltk.download('wordnet')
nltk.download('omw-1.4')
st = nltk.PorterStemmer()
lm = nltk.WordNetLemmatizer()

stop_words = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [167]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [179]:
def tazartu(text):
    n_zhok = text.replace("\n", "")
    text_tazar = "".join([i for i in n_zhok if i not in string.punctuation])
    token = re.split("\W+", text)
    stop_w = [j.lower() for j in token if j not in stop_words]
    sting = [st.stem(w) for w in stop_w]
    lmize = " ".join([lm.lemmatize(k) for k in sting])
    return lmize

df["comment_new"] = df["comment"].apply(lambda x: tazartu(x))
df

,comment,toxic,comment_new
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,верблюдов дебилы бл
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,хохлы это отдушина затюканого россиянина мол в...
2,Собаке - собачья смерть\n,1.0,собаке собачья смерть
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,страницу обнови дебил это оскорбление доказанн...
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,убедил 6 страничный пдф скрипалей отравила рос...
...,...,...,...
14407,Вонючий совковый скот прибежал и ноет. А вот и...,1.0,вонючий совковый скот прибежал ноет а сторонни...
14408,А кого любить? Гоблина тупорылого что-ли? Или ...,1.0,а кого любить гоблина тупорылого или какую про...
14409,"Посмотрел Утомленных солнцем 2. И оказалось, ч...",0.0,посмотрел утомленных солнцем 2 и оказалось это...
14410,КРЫМОТРЕД НАРУШАЕТ ПРАВИЛА РАЗДЕЛА Т.К В НЕМ Н...,1.0,крымотред нарушает правила раздела т к в нем н...


# Векторизация слов

In [169]:
from sklearn.feature_extraction.text import CountVectorizer

In [174]:
vect = CountVectorizer(analyzer=tazartu)
X = vect.fit_transform(df["comment"])

In [175]:
X.shape

(14412, 68440)

In [180]:
ngram = CountVectorizer(ngram_range=(2,2))
X_c = ngram.fit_transform(df["comment_new"])

In [181]:
X_c.shape

(14412, 224508)

# TF-IDF и Обучение данных

In [183]:
def clean(text):
    n_zhok = text.replace("\n", "")
    text_tazar = "".join([i for i in n_zhok if i not in string.punctuation])
    token = re.split("\W+", text)
    stop_w = [j.lower() for j in token if j not in stop_words]
    sting = " ".join([st.stem(w) for w in stop_w])
    return sting

In [210]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [186]:
tf = TfidfVectorizer(analyzer=clean)
X_tf = tf.fit_transform(df["comment"])

In [187]:
X_tf.shape

(14412, 70)

In [190]:
X_tf

<14412x70 sparse matrix of type '<class 'numpy.float64'>'
	with 344553 stored elements in Compressed Sparse Row format>

In [196]:
X = df['comment_new']
y = df['toxic']

In [200]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

svm_mod = make_pipeline(TfidfVectorizer(), SVC(kernel='linear', probability=True))
svm_mod.fit(X_train, y_train)
pred_svm = svm_mod.predict(X_test)
print("SVM accuracy ",metrics.accuracy_score(y_test, pred_svm))

SVM accuracy  0.8629899410336455


In [204]:
lgr = make_pipeline(TfidfVectorizer(), LogisticRegression())
lgr.fit(X_train, y_train)
pred_lgr = lgr.predict(X_test)
print("LogisticRegression ",metrics.accuracy_score(y_test, pred_lgr))

LogisticRegression  0.8168574401664932


In [207]:
dtc = make_pipeline(TfidfVectorizer(), DecisionTreeClassifier())
dtc.fit(X_train, y_train)
pred_dtc = dtc.predict(X_test)
print("DecisionTreeClassifier ",metrics.accuracy_score(y_test, pred_dtc))

DecisionTreeClassifier  0.751647589316684


In [208]:
knn = make_pipeline(TfidfVectorizer(), KNeighborsClassifier())
knn.fit(X_train, y_train)
pred_knn = knn.predict(X_test)
print("KNeighborsClassifier ",metrics.accuracy_score(y_test, pred_knn))

KNeighborsClassifier 0.3260492542490461


In [213]:
rfc = make_pipeline(TfidfVectorizer(), RandomForestClassifier())
rfc.fit(X_train, y_train)
pred_rfc = rfc.predict(X_test)
print('RandomForest ', metrics.accuracy_score(y_test, pred_rfc))

RandomForest  0.8074921956295525


In [220]:
import joblib
with open('model_svm.joblib', 'wb') as file:
    joblib.dump(svm_mod, file)

In [221]:
svcc = joblib.load("model_svm.joblib")
test = "Мирас еблан"
a = tazartu(test)
svcc.predict([a])

array([1.])